In [78]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

In [79]:
filename = 'cleaned_fraudTrain.csv'

# import Dataset
path = '../dataset/midterm_dataset/'+filename
df = pd.read_csv(path)

row_num = df.shape[0]
feature_num = df.shape[1]

print(df.shape)
df

(1296675, 14)


category     amt  is_male      lat      long  city_pop  \
0             misc_net    4.97        0  36.0788  -81.1781      3495   
1          grocery_pos  107.23        0  48.8878 -118.2105       149   
2        entertainment  220.11        1  42.1808 -112.2620      4154   
3        gas_transport   45.00        1  46.2306 -112.1138      1939   
4             misc_pos   41.96        1  38.4207  -79.4629        99   
...                ...     ...      ...      ...       ...       ...   
1296670  entertainment   15.56        1  37.7175 -112.4777       258   
1296671    food_dining   51.70        1  39.2667  -77.5101       100   
1296672    food_dining  105.93        1  32.9396 -105.8189       899   
1296673    food_dining   74.90        1  43.3526 -102.5411      1126   
1296674    food_dining    4.30        1  45.8433 -113.8748       218   

                                       job   unix_time  is_fraud  \
0                Psychologist, counselling  1325376018         0   
1        Special educational needs teacher  1325376044         0   
2              Nature conservation officer  1325376051         0   
3                          Patent attorney  1325376076         0   
4           Dance movement psychotherapist  1325376186         0   
...                                    ...         ...       ...   
1296670                       Geoscientist  1371816728         0   
1296671   Production assistant, television  1371816739         0   
1296672                    Naval architect  1371816752         0   
1296673              Volunteer coordinator  1371816816         0   
1296674           Therapist, horticultural  1371816817         0   

                                  state_city  age  trans_time  trans_year  \
0                  28654, Moravian Falls, NC   31  Late Night        2019   
1                          99160, Orient, WA   41  Late Night        2019   
2                      83252, Malad City, ID   57  Late Night        2019   
3                         59632, Boulder, MT   52  Late Night        2019   
4                        24433, Doe Hill, VA   33  Late Night        2019   
...                                      ...  ...         ...         ...   
1296670                     84735, Hatch, UT   59     Morning        2020   
1296671                 21790, Tuscarora, MD   41     Morning        2020   
1296672  88325, High Rolls Mountain Park, NM   53     Morning        2020   
1296673                 57756, Manderson, SD   40     Morning        2020   
1296674                      59871, Sula, MT   25     Morning        2020   

         trans_month  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  
...              ...  
1296670            6  
1296671            6  
1296672            6  
1296673            6  
1296674            6  

[1296675 rows x 14 columns]

<h2> Transform for Multiple linear regression

In [80]:
#from information gain job and state_city dont have much value so i drop it
#delete all neccesary columns
del_col = ['job', 'state_city']
for col in del_col:
    if col in df.columns.tolist():
        df = df.drop(col, 1)
df.head()

category     amt  is_male      lat      long  city_pop   unix_time  \
0       misc_net    4.97        0  36.0788  -81.1781      3495  1325376018   
1    grocery_pos  107.23        0  48.8878 -118.2105       149  1325376044   
2  entertainment  220.11        1  42.1808 -112.2620      4154  1325376051   
3  gas_transport   45.00        1  46.2306 -112.1138      1939  1325376076   
4       misc_pos   41.96        1  38.4207  -79.4629        99  1325376186   

   is_fraud  age  trans_time  trans_year  trans_month  
0         0   31  Late Night        2019            1  
1         0   41  Late Night        2019            1  
2         0   57  Late Night        2019            1  
3         0   52  Late Night        2019            1  
4         0   33  Late Night        2019            1

<h2> Normalize

In [81]:
nor_col = ['amt', 'city_pop','lat', 'long', 'unix_time', 'age']
        
df[nor_col] = df[nor_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [82]:
df.head()

category       amt  is_male       lat      long  city_pop  \
0       misc_net  0.000137        0  0.343968  0.864638  0.001194   
1    grocery_pos  0.003670        0  0.618450  0.485682  0.000043   
2  entertainment  0.007569        1  0.474727  0.546553  0.001421   
3  gas_transport  0.001520        1  0.561509  0.548070  0.000659   
4       misc_pos  0.001415        1  0.394153  0.882190  0.000026   

      unix_time  is_fraud       age  trans_time  trans_year  trans_month  
0  0.000000e+00         0  0.207317  Late Night        2019            1  
1  5.598526e-07         0  0.329268  Late Night        2019            1  
2  7.105821e-07         0  0.524390  Late Night        2019            1  
3  1.248902e-06         0  0.463415  Late Night        2019            1  
4  3.617509e-06         0  0.231707  Late Night        2019            1

<h2>Get dummy

In [83]:
df = pd.get_dummies(df, columns=['category', 'trans_year', 'trans_month', 'trans_time'])

In [84]:
label = 'is_fraud'
label

'is_fraud'

In [85]:
features = df.loc[:, df.columns != label].columns.tolist()

In [86]:
y_train = df[label]
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [87]:
X_train = df[features]
X_train.head()

amt  is_male       lat      long  city_pop     unix_time       age  \
0  0.000137        0  0.343968  0.864638  0.001194  0.000000e+00  0.207317   
1  0.003670        0  0.618450  0.485682  0.000043  5.598526e-07  0.329268   
2  0.007569        1  0.474727  0.546553  0.001421  7.105821e-07  0.524390   
3  0.001520        1  0.561509  0.548070  0.000659  1.248902e-06  0.463415   
4  0.001415        1  0.394153  0.882190  0.000026  3.617509e-06  0.231707   

   category_entertainment  category_food_dining  category_gas_transport  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       1                     0                       0   
3                       0                     0                       1   
4                       0                     0                       0   

   category_grocery_net  category_grocery_pos  category_health_fitness  \
0                     0                     0                        0   
1                     0                     1                        0   
2                     0                     0                        0   
3                     0                     0                        0   
4                     0                     0                        0   

   category_home  category_kids_pets  category_misc_net  category_misc_pos  \
0              0                   0                  1                  0   
1              0                   0                  0                  0   
2              0                   0                  0                  0   
3              0                   0                  0                  0   
4              0                   0                  0                  1   

   category_personal_care  category_shopping_net  category_shopping_pos  \
0                       0                      0                      0   
1                       0                      0                      0   
2                       0                      0                      0   
3                       0                      0                      0   
4                       0                      0                      0   

   category_travel  trans_year_2019  trans_year_2020  trans_month_1  \
0                0                1                0              1   
1                0                1                0              1   
2                0                1                0              1   
3                0                1                0              1   
4                0                1                0              1   

   trans_month_2  trans_month_3  trans_month_4  trans_month_5  trans_month_6  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   trans_month_7  trans_month_8  trans_month_9  trans_month_10  \
0              0              0              0               0   
1              0              0              0               0   
2              0              0              0               0   
3              0              0              0               0   
4              0              0              0               0   

   trans_month_11  trans_month_12  trans_time_Early Morning  trans_time_Eve  \
0               0               0                         0               0   
1               0               0                         0               0   
2               0               0                         0               0   
3               0               0                         0               0   
4               0      

In [88]:
X_train.shape[1]

41

In [89]:
X_train = X_train.values
y_train = y_train.values
m = len(y_train)

for i in range(2):
    print('x =', X_train[i, ], ', y =', y_train[i])


x = [1.37142936e-04 0.00000000e+00 3.43968440e-01 8.64638464e-01
 1.19449117e-03 0.00000000e+00 2.07317073e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00] , y = 0
x = [3.66969625e-03 0.00000000e+00 6.18449756e-01 4.85681832e-01
 4.33484697e-05 5.59852556e-07 3.29268293e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 1.0000

In [90]:
# Lets use hstack() function from numpy to add column of ones to X feature 
# This will be our final X matrix (feature matrix)
X_train = np.hstack((np.ones((m,1)), X_train ))
X_train[:2]

array([[1.00000000e+00, 1.37142936e-04, 0.00000000e+00, 3.43968440e-01,
        8.64638464e-01, 1.19449117e-03, 0.00000000e+00, 2.07317073e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 3.66969625e-03, 0.00000000e+00, 6.18449756e-01,
        4.85681832e-01, 4.33484697e-05, 5.59852556e-07, 3.29268293e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00,

In [91]:
def compute_cost(X, y, theta):
    """
    Compute the cost of a particular choice of theta for linear regression.

    Input Parameters
    ----------------
    X : 2D array where each row represent the training example and each column represent the feature ndarray. Dimension(m x n)
        m= number of training examples
        n= number of features (including X_0 column of ones)
    y : 1D array of labels/target value for each traing example. dimension(1 x m)

    theta : 1D array of fitting parameters or weights. Dimension (1 x n)

    Output Parameters
    -----------------
    J : Scalar value.
    """
    predictions = X.dot(theta)
      #print('predictions= ', predictions[:5])
    errors = np.subtract(predictions, y)
      #print('errors= ', errors[:5]) 
    sqrErrors = np.square(errors)
      #print('sqrErrors= ', sqrErrors[:5]) 
      #J = 1 / (2 * m) * np.sum(sqrErrors)
      # OR
      # We can merge 'square' and 'sum' into one by taking the transpose of matrix 'errors' and taking dot product with itself
      # If your confuse about this try to do this with few values for better understanding  
    J = 1/(2 * m) * errors.T.dot(errors)

    return J

In [92]:
def gradient_descent(X, y, theta, alpha, iterations):
    """
    Compute cost for linear regression.
  
    Input Parameters
    ----------------
    X : 2D array where each row represent the training example and each column represent the feature ndarray. Dimension(m x n)
        m= number of training examples
        n= number of features (including X_0 column of ones)
    y : 1D array of labels/target value for each traing example. dimension(m x 1)
    theta : 1D array of fitting parameters or weights. Dimension (1 x n)
    alpha : Learning rate. Scalar value
    iterations: No of iterations. Scalar value. 

    Output Parameters
    -----------------
    theta : Final Value. 1D array of fitting parameters or weights. Dimension (1 x n)
    cost_history: Conatins value of cost for each iteration. 1D array. Dimansion(m x 1)
    """
    cost_history = np.zeros(iterations)

    for i in range(iterations):
        predictions = X.dot(theta)
        #print('predictions= ', predictions[:5])
        errors = np.subtract(predictions, y)
        #print('errors= ', errors[:5])
        sum_delta = (alpha / m) * X.transpose().dot(errors);
        #print('sum_delta= ', sum_delta[:5])
        theta = theta - sum_delta;

    cost_history[i] = compute_cost(X, y, theta)  
  
    return theta, cost_history

In [93]:
#set hyper parameter
theta = np.zeros(42)
iterations = 500;
alpha = 0.15;

In [94]:
theta, cost_history = gradient_descent(X_train, y_train, theta, alpha, iterations)
print('Final value of theta =', theta)
print('First 5 values from cost_history =', cost_history[:5])
print('Last 5 values from cost_history =', cost_history[-5 :])

Final value of theta = [ 7.55320962e-04  6.65711697e-03  1.62453578e-03  5.43972944e-04
  6.97311749e-04  1.27610474e-03 -1.01980585e-03  7.67522800e-03
 -4.86557132e-03 -6.12706792e-03  4.14292603e-03  2.09176875e-03
  1.32946293e-02 -7.10987858e-03 -7.30240093e-03 -6.77963785e-03
  1.17107284e-02 -8.46435039e-04 -6.23204900e-03  1.21703337e-02
  1.73733708e-03 -5.12936166e-03  3.72856411e-04  3.82464551e-04
  2.16197764e-03  2.82436338e-03  6.98198685e-04 -7.54290127e-04
  6.22118347e-04 -8.67303560e-04 -2.00141637e-03 -1.36869610e-03
  1.40429710e-04  8.03180990e-04 -1.01481662e-04 -1.40175997e-03
 -1.05184704e-02 -3.62567233e-05  2.67729561e-04 -7.15020557e-03
  1.81606397e-02  3.18843430e-05]
First 5 values from cost_history = [0. 0. 0. 0. 0.]
Last 5 values from cost_history = [0.        0.        0.        0.        0.0028288]


In [95]:
theta

array([ 7.55320962e-04,  6.65711697e-03,  1.62453578e-03,  5.43972944e-04,
        6.97311749e-04,  1.27610474e-03, -1.01980585e-03,  7.67522800e-03,
       -4.86557132e-03, -6.12706792e-03,  4.14292603e-03,  2.09176875e-03,
        1.32946293e-02, -7.10987858e-03, -7.30240093e-03, -6.77963785e-03,
        1.17107284e-02, -8.46435039e-04, -6.23204900e-03,  1.21703337e-02,
        1.73733708e-03, -5.12936166e-03,  3.72856411e-04,  3.82464551e-04,
        2.16197764e-03,  2.82436338e-03,  6.98198685e-04, -7.54290127e-04,
        6.22118347e-04, -8.67303560e-04, -2.00141637e-03, -1.36869610e-03,
        1.40429710e-04,  8.03180990e-04, -1.01481662e-04, -1.40175997e-03,
       -1.05184704e-02, -3.62567233e-05,  2.67729561e-04, -7.15020557e-03,
        1.81606397e-02,  3.18843430e-05])

In [96]:
fraud_counter = 0
for i in y_train:
    if i == 1:
        fraud_counter += 1
fraud_counter/len(y_train)

0.005788651743883394

<h1>import test data

In [118]:

filename = 'cleaned_fraudTest.csv'

# import Dataset
path = '../dataset/midterm_dataset/'+filename
df_test = pd.read_csv(path)

row_num = df_test.shape[0]
feature_num = df_test.shape[1]

print(df.shape)
df_test.head()

(1296675, 42)


category    amt  is_male      lat      long  city_pop  \
0   personal_care   2.86        1  33.9659  -80.9355    333497   
1   personal_care  29.84        0  40.3207 -110.4360       302   
2  health_fitness  41.28        0  40.6729  -73.5365     34496   
3        misc_pos  60.05        1  28.5697  -80.8191     54767   
4          travel   3.19        1  44.2529  -85.0170      1126   

                      job   unix_time  is_fraud             state_city  age  \
0     Mechanical engineer  1371816865         0    29209, Columbia, SC   52   
1  Sales professional, IT  1371816873         0     84002, Altonah, UT   30   
2       Librarian, public  1371816893         0    11710, Bellmore, NY   50   
3            Set designer  1371816915         0  32780, Titusville, FL   33   
4      Furniture designer  1371816917         0    49632, Falmouth, MI   65   

  trans_time  trans_year  trans_month  
0    Morning        2020            6  
1    Morning        2020            6  
2    Morning        2020            6  
3    Morning        2020            6  
4    Morning        2020            6

<h2> Transform for Multiple linear regression

In [119]:
#from information gain job and state_city dont have much value so i drop it
#delete all neccesary columns
del_col = ['job', 'state_city']
for col in del_col:
    if col in df_test.columns.tolist():
        df_test = df_test.drop(col, 1)
df_test.head()

category    amt  is_male      lat      long  city_pop   unix_time  \
0   personal_care   2.86        1  33.9659  -80.9355    333497  1371816865   
1   personal_care  29.84        0  40.3207 -110.4360       302  1371816873   
2  health_fitness  41.28        0  40.6729  -73.5365     34496  1371816893   
3        misc_pos  60.05        1  28.5697  -80.8191     54767  1371816915   
4          travel   3.19        1  44.2529  -85.0170      1126  1371816917   

   is_fraud  age trans_time  trans_year  trans_month  
0         0   52    Morning        2020            6  
1         0   30    Morning        2020            6  
2         0   50    Morning        2020            6  
3         0   33    Morning        2020            6  
4         0   65    Morning        2020            6

<h2> Normalize

In [120]:
nor_col = ['amt', 'city_pop','lat', 'long', 'unix_time', 'age']
        
df_test[nor_col] = df_test[nor_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [121]:
df_test.head()

category       amt  is_male       lat      long  city_pop  \
0   personal_care  0.000082        1  0.305255  0.867121  0.114727   
1   personal_care  0.001267        0  0.444423  0.565239  0.000096   
2  health_fitness  0.001769        0  0.452136  0.942836  0.011860   
3        misc_pos  0.002594        1  0.187080  0.868312  0.018834   
4          travel  0.000096        1  0.530537  0.825355  0.000379   

      unix_time  is_fraud       age trans_time  trans_year  trans_month  
0  0.000000e+00         0  0.456790    Morning        2020            6  
1  4.785402e-07         0  0.185185    Morning        2020            6  
2  1.674891e-06         0  0.432099    Morning        2020            6  
3  2.990876e-06         0  0.222222    Morning        2020            6  
4  3.110511e-06         0  0.617284    Morning        2020            6

<h2>Get dummy

In [122]:
df_test = pd.get_dummies(df_test, columns=['category', 'trans_year', 'trans_month', 'trans_time'])

In [123]:
df_test.head()

amt  is_male       lat      long  city_pop     unix_time  is_fraud  \
0  0.000082        1  0.305255  0.867121  0.114727  0.000000e+00         0   
1  0.001267        0  0.444423  0.565239  0.000096  4.785402e-07         0   
2  0.001769        0  0.452136  0.942836  0.011860  1.674891e-06         0   
3  0.002594        1  0.187080  0.868312  0.018834  2.990876e-06         0   
4  0.000096        1  0.530537  0.825355  0.000379  3.110511e-06         0   

        age  category_entertainment  category_food_dining  \
0  0.456790                       0                     0   
1  0.185185                       0                     0   
2  0.432099                       0                     0   
3  0.222222                       0                     0   
4  0.617284                       0                     0   

   category_gas_transport  category_grocery_net  category_grocery_pos  \
0                       0                     0                     0   
1                       0                     0                     0   
2                       0                     0                     0   
3                       0                     0                     0   
4                       0                     0                     0   

   category_health_fitness  category_home  category_kids_pets  \
0                        0              0                   0   
1                        0              0                   0   
2                        1              0                   0   
3                        0              0                   0   
4                        0              0                   0   

   category_misc_net  category_misc_pos  category_personal_care  \
0                  0                  0                       1   
1                  0                  0                       1   
2                  0                  0                       0   
3                  0                  1                       0   
4                  0                  0                       0   

   category_shopping_net  category_shopping_pos  category_travel  \
0                      0                      0                0   
1                      0                      0                0   
2                      0                      0                0   
3                      0                      0                0   
4                      0                      0                1   

   trans_year_2020  trans_month_6  trans_month_7  trans_month_8  \
0                1              1              0              0   
1                1              1              0              0   
2                1              1              0              0   
3                1              1              0              0   
4                1              1              0              0   

   trans_month_9  trans_month_10  trans_month_11  trans_month_12  \
0              0               0               0               0   
1              0               0               0               0   
2              0               0               0               0   
3              0               0               0               0   
4              0               0               0               0   

   trans_time_Early Morning  trans_time_Eve  trans_time_Late Night  \
0                         0               0                      0   
1                         0               0                      0   
2                         0               0                      0   
3                         0               0                      0   
4                         0               0                      0   

   trans_time_Morning  trans_time_Night  trans_time_Noon  
0                   1                 0                0  
1                   1                 0                0  
2                   1                 0                0  
3                   1                 0                0  
4 

In [174]:
y_test = df_test[label]
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [125]:
t_features = df_test.loc[:, df_test.columns != label].columns.tolist()

In [175]:
X_test = df_test[t_features]
X_test.head()

amt  is_male       lat      long  city_pop     unix_time       age  \
0  0.000082        1  0.305255  0.867121  0.114727  0.000000e+00  0.456790   
1  0.001267        0  0.444423  0.565239  0.000096  4.785402e-07  0.185185   
2  0.001769        0  0.452136  0.942836  0.011860  1.674891e-06  0.432099   
3  0.002594        1  0.187080  0.868312  0.018834  2.990876e-06  0.222222   
4  0.000096        1  0.530537  0.825355  0.000379  3.110511e-06  0.617284   

   category_entertainment  category_food_dining  category_gas_transport  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       0                     0                       0   
3                       0                     0                       0   
4                       0                     0                       0   

   category_grocery_net  category_grocery_pos  category_health_fitness  \
0                     0                     0                        0   
1                     0                     0                        0   
2                     0                     0                        1   
3                     0                     0                        0   
4                     0                     0                        0   

   category_home  category_kids_pets  category_misc_net  category_misc_pos  \
0              0                   0                  0                  0   
1              0                   0                  0                  0   
2              0                   0                  0                  0   
3              0                   0                  0                  1   
4              0                   0                  0                  0   

   category_personal_care  category_shopping_net  category_shopping_pos  \
0                       1                      0                      0   
1                       1                      0                      0   
2                       0                      0                      0   
3                       0                      0                      0   
4                       0                      0                      0   

   category_travel  trans_year_2020  trans_month_6  trans_month_7  \
0                0                1              1              0   
1                0                1              1              0   
2                0                1              1              0   
3                0                1              1              0   
4                1                1              1              0   

   trans_month_8  trans_month_9  trans_month_10  trans_month_11  \
0              0              0               0               0   
1              0              0               0               0   
2              0              0               0               0   
3              0              0               0               0   
4              0              0               0               0   

   trans_month_12  trans_time_Early Morning  trans_time_Eve  \
0               0                         0               0   
1               0                         0               0   
2               0                         0               0   
3               0                         0               0   
4               0                         0               0   

   trans_time_Late Night  trans_time_Morning  trans_time_Night  \
0                      0                   1                 0   
1                      0                   1                 0   
2                      0                   1                 0   
3                      0                   1                 0   
4                      0                   1                 0   

   trans_time_Noon  
0                0  
1                0  
2                0  
3                0  
4                0

In [127]:
X_test.shape[1]

35

In [176]:
X_test = X_test.values
y_test = y_test.values
m = len(y_test)

for i in range(2):
    print('x_test =', X_test[i, ], ', y_test =', y_test[i])


x_test = [8.16967986e-05 1.00000000e+00 3.05255043e-01 8.67121017e-01
 1.14726886e-01 0.00000000e+00 4.56790123e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00] , y_test = 0
x_test = [1.26673961e-03 0.00000000e+00 4.44423031e-01 5.65239148e-01
 9.59858973e-05 4.78540194e-07 1.85185185e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.0000

In [178]:
print(len(t_features))

35


In [179]:
print(len(features))

41


In [180]:
#check which not in
not_in_test = [i for i in features if i not in t_features]
not_in_test

['trans_year_2019',
 'trans_month_1',
 'trans_month_2',
 'trans_month_3',
 'trans_month_4',
 'trans_month_5']

In [181]:
#check index
index_not_in = [features.index(i) for i in not_in_test]
index_not_in

[21, 23, 24, 25, 26, 27]

In [182]:
test_theta = theta
for i in sorted(index_not_in, reverse=True):
    test_theta = np.delete(test_theta, i)
len(test_theta)

36

In [183]:
y_pred = []
for i in range(len(X_test)):
    test_data = np.hstack((np.ones(1), X_test[i,]))
    y_pred.append(test_data.dot(test_theta))

In [187]:
len(y_pred), len(y_test)

(555719, 555719)

<h1> Evaluation

<h2> Mean Absolute Error (MAE)

In [216]:
difference = []
zip_object = zip(y_pred, y_test)
for y_pred_i, y_test_i in zip_object:
    difference.append(abs(y_pred_i- y_test_i))

# Mean Absolute Error (MAE)
mae = 1/len(y_pred) * np.sum(difference)
mae

0.011809592195574032

<h2> Mean Squared Error (MSE)

In [222]:
diff_power= []
zip_object = zip(y_pred, y_test)
for y_pred_i, y_test_i in zip_object:
    diff_power.append((y_pred_i- y_test_i)**2)

# Mean Squared Error (MSE)
mse = 1/len(y_pred) * np.sum(diff_power)
mse

0.0038140652997438766

<h2> Root Mean Squared Error (RMSE)

In [223]:
# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
rmse

0.06175811930219278